In [ ]:
!pip install kfp


In [ ]:
import kfp
from kfp import dsl

In [ ]:
import os
import sys
sys.path.append('../lib')

# Convert notebooks to python

In [ ]:
from igz import v3io_cred, xcp_op, mount_v3io

In [ ]:
NOTEBOOKS_DIR = os.getenv('NOTEBOOKS_DIR','/User/GitHub/igxartifacts/location_based_recommendation')
PREP_NB = os.getenv('PREP_NB','02-preprocessing.ipynb')
TRAIN_NB = os.getenv('TRAIN_NB','03-train_prod2vec_skipgram.ipynb')
LGBM_NB = os.getenv('LGBM_NB','04-lgbm.ipynb')
WORK_DIR = os.getenv('WORK_DIR','/v3io/bigdata/lbr')
LBR_PATH  = os.getenv('LBR_PATH','/v3io/bigdata/lbr')

In [ ]:
print(NOTEBOOKS_DIR)

In [ ]:
%env NOTEBOOKS_DIR={NOTEBOOKS_DIR}
%env PREP_NB={PREP_NB}
%env TRAIN_NB={TRAIN_NB}
%env LGBM_NB={LGBM_NB}

In [ ]:
!jupyter nbconvert --output-dir={WORK_DIR} --to python {NOTEBOOKS_DIR}/{PREP_NB}
!jupyter nbconvert --output-dir={WORK_DIR} --to python {NOTEBOOKS_DIR}/{TRAIN_NB}
!jupyter nbconvert --output-dir={WORK_DIR} --to python {NOTEBOOKS_DIR}/{LGBM_NB}

#!echo {NOTEBOOKS_DIR}/{PREP_NB}

In [ ]:
!cp 

In [ ]:
nuclio_deploy = kfp.components.load_component(url='https://raw.githubusercontent.com/kubeflow/pipelines/master/components/nuclio/deploy/component.yaml')
nuclio_invoke = kfp.components.load_component(url='https://raw.githubusercontent.com/kubeflow/pipelines/master/components/nuclio/invoker/component.yaml')

In [ ]:
def run_step(NAME,CMD,LBR_PATH):
    """Print env vars."""
    cop = dsl.ContainerOp(
        name=NAME,
        image='python:3.6-jessie',
        command=['ls /lbr',CMD],
    )
    cop.apply(v3io_cred())
    cop.apply(mount_v3io(container='bigdata', sub_path='/lbr/01-generate_stores_and_customers/ipynb', mount_path='/lbr'))
    return cop

In [ ]:
@dsl.pipeline(
    name='LBR pipeline',
    description='Runnign LBR in kubeflow'
)
def my_pipeline(
   txt='good morningf',
   val=7,
):
    #step1 = run_step('PREP','python /lbr/02-preprocessing.py',LBR_PATH)
    build = nuclio_deploy(url='/lbr/', name='myfunc', project='myproj', tag='0.11', dashboard='http://default-tenant-nuclio-dashboard:8070').after(train)

    

In [ ]:
kfp.compiler.Compiler().compile(my_pipeline, 'LBRpipe.yaml')

In [ ]:
EXPERIMENT_NAME = 'LBR pipeline'

In [ ]:
client = kfp.Client(namespace='default-tenant')
arguments = {'txt': 'good evening', 'val': 7}
experiment = client.create_experiment(EXPERIMENT_NAME)
run_name = my_pipeline.__name__ + ' run'
run_result = client.run_pipeline(experiment.id, run_name, 'LBRpipe.yaml', arguments)